# Goodreads Books Reviews

#### Imports

In [23]:
import numpy as np
import pandas as pd

from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, TrainingArguments, Trainer
from datasets import load_dataset, load_metric
from huggingface_hub import notebook_login

import torch

#### Load Data

In [24]:
goodreads_train = load_dataset('csv', data_files='./data/goodreads_train.csv')['train']
goodreads_test = load_dataset('csv', data_files='./data/goodreads_test.csv')['train']

Using custom data configuration default-8d12ff12b322328a
Reusing dataset csv (C:\Users\Terence.Yeung\.cache\huggingface\datasets\csv\default-8d12ff12b322328a\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-ef13ee0a229a23e6
Reusing dataset csv (C:\Users\Terence.Yeung\.cache\huggingface\datasets\csv\default-ef13ee0a229a23e6\0.0.0\433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

#### Exploratory Data Analysis

In [25]:
# Easier to inspect with pandas

goodreads_train_df = goodreads_train.to_pandas()
goodreads_test_df = goodreads_test.to_pandas()

In [26]:
goodreads_train_df.head()

,user_id,book_id,review_id,rating,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,8842281e1d1347389f2ab93d60773d4d,18245960,dfdbb7b0eb5a7e4c26d59a937e2e5feb,5,This is a special book. It started slow for ab...,Sun Jul 30 07:44:10 -0700 2017,Wed Aug 30 00:00:26 -0700 2017,Sat Aug 26 12:05:52 -0700 2017,Tue Aug 15 13:23:18 -0700 2017,28,1
1,8842281e1d1347389f2ab93d60773d4d,16981,a5d2c3628987712d0e05c4f90798eb67,3,Recommended by Don Katz. Avail for free in Dec...,Mon Dec 05 10:46:44 -0800 2016,Wed Mar 22 11:37:04 -0700 2017,None,None,1,0
2,8842281e1d1347389f2ab93d60773d4d,28684704,2ede853b14dc4583f96cf5d120af636f,3,"A fun, fast paced science fiction thriller. I ...",Tue Nov 15 11:29:22 -0800 2016,Mon Mar 20 23:40:27 -0700 2017,Sat Mar 18 23:22:42 -0700 2017,Fri Mar 17 23:45:40 -0700 2017,22,0
3,8842281e1d1347389f2ab93d60773d4d,27161156,ced5675e55cd9d38a524743f5c40996e,0,Recommended reading to understand what is goin...,Wed Nov 09 17:37:04 -0800 2016,Wed Nov 09 17:38:20 -0800 2016,None,None,5,1
4,8842281e1d1347389f2ab93d60773d4d,25884323,332732725863131279a8e345b63ac33e,4,"I really enjoyed this book, and there is a lot...",Mon Apr 25 09:31:23 -0700 2016,Mon Apr 25 09:31:23 -0700 2016,Sun Jun 26 00:00:00 -0700 2016,Sat May 28 00:00:00 -0700 2016,9,1


In [27]:
goodreads_test_df.head()

,user_id,book_id,review_id,review_text,date_added,date_updated,read_at,started_at,n_votes,n_comments
0,b9450d1c1f97f891c392b1105959b56e,7092507,5c4df7e70e9b438c761f07a4620ccb7c,** spoiler alert ** \n This is definitely one ...,Sat Nov 10 06:06:13 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sun Nov 11 05:38:36 -0800 2012,Sat Nov 10 00:00:00 -0800 2012,1,0
1,b9450d1c1f97f891c392b1105959b56e,5576654,8eaeaf13213eeb16ad879a2a2591bbe5,"** spoiler alert ** \n ""You are what you drink...",Fri Nov 09 21:55:16 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Sat Nov 10 05:41:49 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,1,0
2,b9450d1c1f97f891c392b1105959b56e,15754052,dce649b733c153ba5363a0413cac988f,Roar is one of my favorite characters in Under...,Fri Nov 09 00:25:50 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Sat Nov 10 06:14:10 -0800 2012,Fri Nov 09 00:00:00 -0800 2012,0,0
3,b9450d1c1f97f891c392b1105959b56e,17020,8a46df0bb997269d6834f9437a4b0a77,** spoiler alert ** \n If you feel like travel...,Thu Nov 01 00:28:39 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Sat Nov 03 11:35:22 -0700 2012,Thu Nov 01 00:00:00 -0700 2012,0,0
4,b9450d1c1f97f891c392b1105959b56e,12551082,d11d3091e22f1cf3cb865598de197599,3.5 stars \n I read and enjoyed the first two ...,Thu Oct 18 00:57:00 -0700 2012,Mon Apr 01 23:00:51 -0700 2013,Sat Mar 30 00:00:00 -0700 2013,Fri Mar 29 00:00:00 -0700 2013,0,0


In [28]:
goodreads_train_df['rating'].value_counts().sort_index()

0     30988
1     28718
2     72627
3    188972
4    313688
5    265007
Name: rating, dtype: int64

#### Preprocessing

In [29]:
# Dropping all columns except review_text and rating

dropped_columns = ['user_id', 'book_id', 'review_id', 'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes', 'n_comments']

goodreads_train = goodreads_train.remove_columns(dropped_columns)
goodreads_test = goodreads_test.remove_columns(dropped_columns)

# Renaming text and label columns

goodreads_train = goodreads_train.rename_column('review_text', 'text')
goodreads_train = goodreads_train.rename_column('rating', 'label')
goodreads_test = goodreads_test.rename_column('review_text', 'text')

In [30]:
goodreads_train

Dataset({
    features: ['label', 'text'],
    num_rows: 900000
})

In [31]:
goodreads_test

Dataset({
    features: ['text'],
    num_rows: 478033
})

In [32]:
# Splitting training data into a training and validation set as test dataset provided is not labelled

goodreads_train_split = goodreads_train.train_test_split(test_size=0.2)

In [33]:
goodreads_train_split

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 720000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 180000
    })
})

In [34]:
# Taking only a small sample of the training data due to computational limitations

def reduce_dataset(dataset):
    total_rows = dataset.num_rows
    sample_size = int(total_rows * 0.001)
    sample_indices = np.random.RandomState(42).randint(total_rows, size=sample_size)
    return dataset.select(sample_indices)

train_data = reduce_dataset(goodreads_train_split['train'])
test_data = reduce_dataset(goodreads_train_split['test'])

In [35]:
train_data

Dataset({
    features: ['label', 'text'],
    num_rows: 720
})

In [36]:
test_data

Dataset({
    features: ['label', 'text'],
    num_rows: 180
})

In [37]:
# Labels need to be encoded to ClassLabel type for custom datasets

train_data = train_data.class_encode_column('label')

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [38]:
train_data.features

{'label': ClassLabel(num_classes=6, names=['0', '1', '2', '3', '4', '5'], id=None),
 'text': Value(dtype='string', id=None)}

In [39]:
test_data = test_data.class_encode_column('label')

Stringifying the column:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting to class labels:   0%|          | 0/1 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/1 [00:00<?, ?ba/s]

In [40]:
test_data.features

{'label': ClassLabel(num_classes=6, names=['0', '1', '2', '3', '4', '5'], id=None),
 'text': Value(dtype='string', id=None)}

In [45]:
# Tokenizing review text

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize_text(data):
    return tokenizer(data['text'], truncation=True)
 
tokenized_train = train_data.map(tokenize_text, batched=True)
tokenized_test = test_data.map(tokenize_text, batched=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [46]:
# Convert training samples to PyTorch tensors and concatenate them with the correct amount of padding to speed up training

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length', return_tensors='pt')

In [47]:
tokenized_train

Dataset({
    features: ['label', 'text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 720
})

#### Model Building

In [48]:
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [49]:
# Defining metrics

def compute_metrics(eval_pred):
    load_accuracy = load_metric('accuracy')
    load_precision = load_metric('precision')
    load_recall = load_metric('recall')
    load_f1 = load_metric('f1')
  
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)['accuracy']
    precision = load_precision.compute(predictions=predictions, references=labels)['precision']
    recall = load_recall.compute(predictions=predictions, references=labels)['recall']
    f1 = load_f1.compute(predictions=predictions, references=labels)["f1"]
    
    return {'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}

#### Model Training

In [50]:
# Training hyperparameters

training_args = TrainingArguments(
    output_dir='test_trainer',
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
)

In [51]:
trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
C:\Users\Terence.Yeung\Anaconda3\envs\arup_nlp_prog\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 720
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 225


Epoch,Training Loss,Validation Loss


In [ ]:
trainer.train_dataset[0]